# 1. Title and Description

### From Pickle to TFLite: ASL Hand Gesture Recognition Model

This notebook loads American Sign Language (ASL) hand landmark data from a `.pickle` file, trains a neural network classifier, and exports the model in both TensorFlow (`.h5`) and TensorFlow Lite (`.tflite`) formats. It also saves the label encoder for future use in real-time gesture recognition applications.


# 2. Imports

In [ ]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 3. Load Dataset

In [ ]:
# Load the dataset from a pickle file
with open("./ASL.pickle", "rb") as f:
    dataset = pickle.load(f)

# Convert to NumPy arrays
data = np.asarray(dataset["dataset"], dtype=np.float32)
labels = np.asarray(dataset["labels"])

# 4. Encode Labels

In [ ]:
# Encode string labels into integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

# 5. Train-Test Split

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    data, y, test_size=0.2, shuffle=True, stratify=y, random_state=42
)

# 6. Build the Model

In [ ]:
# Define a simple feedforward neural network
model = keras.Sequential([
    keras.layers.Input(shape=(42,)),  # 21 landmarks × 2 (x, y)
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# 7. Compile and Train

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# 8. Save TensorFlow Model

In [ ]:
# Save the trained model
model.save("./ASL_model.h5")

 # 9. Convert to TFLite

In [ ]:
# Convert to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open("./ASL_model.tflite", "wb") as f:
    f.write(tflite_model)


# 10. Save Label Encoder

In [ ]:
# Save the label encoder for future use
with open("./label_encoder.pickle", "wb") as f:
    pickle.dump(label_encoder, f)

# 11. Test Label Encoder Loading (Optional)

In [ ]:
# Load and print stored labels (for verification)
with open("label_encoder2.pickle", "rb") as file:
    label_encoder = pickle.load(file)

print("Classes:", label_encoder.classes_)